<a href="https://colab.research.google.com/github/kakakafafafa888/Adipose_Ghost_Analysis/blob/main/data_spider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# @title 环境准备
# =========================================

# 安装依赖
!pip install -q ipywidgets nest_asyncio matplotlib aiohttp biopython rank_bm25 beautifulsoup4

# 启用 ipywidgets（Colab 必须）
from google.colab import output
output.enable_custom_widget_manager()

import nest_asyncio
nest_asyncio.apply()

from google.colab import drive
import os, re, json, asyncio
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from pathlib import Path

# 挂载 Google Drive
drive.mount('/content/drive')

# 创建项目目录
project_dir = "/content/drive/MyDrive/GEO_Search_Project"
subdirs = ["results", "annotations", "models"]
for sd in subdirs:
    os.makedirs(os.path.join(project_dir, sd), exist_ok=True)

print("📁 项目目录结构已准备完成：")
print(f"   主目录: {project_dir}")
for sd in subdirs:
    print(f"   └── {sd}/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 项目目录结构已准备完成：
   主目录: /content/drive/MyDrive/GEO_Search_Project
   └── results/
   └── annotations/
   └── models/


In [35]:
# @title 工具函数与参数设置
import re, json, asyncio
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import HTML

# --------- 工具函数 ---------

def sanitize_for_filename(text, fallback="results"):
    if not text or not text.strip():
        return fallback
    t = re.sub(r"[^\w\-]+", "_", text.strip())
    t = re.sub(r"_+", "_", t).strip("_")
    return t[:60] if t else fallback

def keywords_to_name(keywords):
    if not keywords:
        return "geo_results"
    joined = "_".join([k.strip() for k in keywords if k.strip()])
    return sanitize_for_filename(joined, fallback="geo_results")

def query_to_name(query):
    return sanitize_for_filename(query, fallback="pubmed_results")

def make_accession_link_column(df, acc_col="accession", link_col="link", new_col="accession_link"):
    """新增一列 accession_link 用于展示超链接"""
    if acc_col in df.columns and link_col in df.columns:
        df[new_col] = df.apply(lambda x: f"<a href='{x[link_col]}' target='_blank'>{x[acc_col]}</a>", axis=1)

def html_display(df, cols=None):
    if cols is None:
        cols = df.columns.tolist()
    return HTML(df[cols].to_html(escape=False, index=False))

# --------- 这里需要你已有的 pubmed_search / geo_keyword_search ---------
# 请确保在此 cell 或之前已经定义了 pubmed_search 和 geo_keyword_search


In [36]:
# @title 可视化搜索与保存
import ipywidgets as widgets
from IPython.display import display, clear_output

# --------- 控件定义 ---------

# PubMed 控件
pm_query = widgets.Text(value='adipose AND macrophage', description='PubMed语句:')
pm_year_start = widgets.IntText(value=2018, description='起始年:')
pm_year_end = widgets.IntText(value=2024, description='结束年:')
pm_sort = widgets.Dropdown(options=[('相关性', 'relevance'), ('按年份', 'pub+date')], value='relevance', description='排序:')
pm_limit = widgets.IntSlider(value=200, min=50, max=500, step=50, description='每页数:')
pm_cap = widgets.IntSlider(value=1000, min=200, max=5000, step=200, description='上限数:')
pm_button = widgets.Button(description='📚 搜索并保存 PubMed', button_style='info')
pm_output = widgets.Output()

# GEO 控件
kw_input = widgets.Text(value='adipose, single-cell, RNA-seq', description='关键词:')
weight_input = widgets.Text(value='adipose:2, RNA-seq:1', description='权重:')
author_input = widgets.Text(value='', description='作者:')
title_input = widgets.Text(value='', description='标题:')
tissue_input = widgets.Text(value='', description='组织:')
geo_year_start = widgets.IntText(value=2018, description='起始年:')
geo_year_end = widgets.IntText(value=2024, description='结束年:')
geo_sort = widgets.Dropdown(options=[('相关性', 'relevance'), ('按年份', 'year')], value='relevance', description='排序:')
geo_limit = widgets.IntSlider(value=200, min=50, max=500, step=50, description='每页数:')
geo_cap = widgets.IntSlider(value=1500, min=200, max=5000, step=200, description='上限数:')
geo_button = widgets.Button(description='🔍 搜索并保存 GEO', button_style='success')
geo_output = widgets.Output()

# --------- PubMed 事件 ---------

def on_pubmed_clicked(b):
    with pm_output:
        clear_output()
        print("🔍 正在整合搜索 PubMed ...")

        year_range = (int(pm_year_start.value), int(pm_year_end.value)) if (pm_year_start.value and pm_year_end.value) else None
        per_page = int(pm_limit.value)
        cap = int(pm_cap.value)
        query_str = pm_query.value.strip()

        if not query_str:
            print("❌ 查询语句为空")
            return

        result = fetch_all_pubmed_results(
            query=query_str,
            year_range=year_range,
            sort_mode=pm_sort.value,
            per_page=per_page,
            max_items=cap
        )
        items, count = result["items"], result["count"]
        print(f"📦 总条目: {count} | 已整合: {len(items)}")

        if not items:
            print("❌ 没有找到相关文献")
            return

        df = pd.DataFrame(items)
        display(df)

        fname = query_to_name(query_str)
        out_dir = Path(f"{project_dir}/results")
        out_dir.mkdir(parents=True, exist_ok=True)
        df.to_csv(out_dir / f"{fname}.csv", index=False)
        with open(out_dir / f"{fname}.json", "w", encoding="utf-8") as f:
            json.dump(items, f, indent=2)
        print(f"💾 已保存到: results/{fname}.csv / .json")

        years = pubmed_years(items)
        if years:
            plt.figure(figsize=(6,3))
            plt.hist(years, bins=range(min(years), max(years)+2), edgecolor='black')
            plt.title("PubMed 结果年份分布")
            plt.xlabel("Year")
            plt.ylabel("Count")
            plt.show()

pm_button.on_click(on_pubmed_clicked)

# --------- GEO 事件 ---------

def on_geo_clicked(b):
    with geo_output:
        clear_output()
        print("🔍 正在整合搜索 GEO 数据集 ...")

        kw_list = [k.strip() for k in kw_input.value.split(",") if k.strip()]
        weights = {}
        for pair in weight_input.value.split(","):
            if ":" in pair:
                k, v = pair.split(":")
                try:
                    weights[k.strip()] = float(v.strip())
                except:
                    pass

        year_range = (int(geo_year_start.value), int(geo_year_end.value)) if (geo_year_start.value and geo_year_end.value) else None
        per_page = int(geo_limit.value)
        cap = int(geo_cap.value)

        result = asyncio.run(fetch_all_geo_results(
            keywords=kw_list,
            author=author_input.value or None,
            title=title_input.value or None,
            tissue=tissue_input.value or None,
            year_range=year_range,
            weights=weights,
            sort_mode=geo_sort.value,
            per_page=per_page,
            max_items=cap
        ))
        items, query_str = result["items"], result["query"]
        print(f"📦 已整合: {len(items)}")
        print(f"🧾 查询语句: {query_str}")

        if not items:
            print("❌ 没有找到 GEO 数据集")
            return

        df = pd.DataFrame(items)
        make_accession_link_column(df, acc_col="accession", link_col="link", new_col="accession_link")

        core_cols = [c for c in ["accession_link","title","taxon","pub_date","update_date","status","overall_design","citation","score"] if c in df.columns]
        display(html_display(df, cols=core_cols))

        fname = keywords_to_name(kw_list)
        out_dir = Path(f"{project_dir}/results")
        out_dir.mkdir(parents=True, exist_ok=True)

        save_cols = [c for c in ["accession","link","title","taxon","pub_date","update_date","status","overall_design","citation","score"] if c in df.columns]
        df[save_cols].to_csv(out_dir / f"{fname}.csv", index=False)
        with open(out_dir / f"{fname}.json", "w", encoding="utf-8") as f:
            json.dump(items, f, indent=2)
        print(f"💾 已保存到: results/{fname}.csv / .json")

geo_button.on_click(on_geo_clicked)

# --------- 显示界面 ---------

display(widgets.VBox([
    widgets.Label("📘 PubMed 文献探索与整合"),
    widgets.HBox([pm_query]),
    widgets.HBox([pm_year_start, pm_year_end, pm_sort]),
    widgets.HBox([pm_limit, pm_cap]),
    pm_button, pm_output,
    widgets.Label("🧬 GEO 数据集搜索与整合"),
    kw_input, weight_input,
    widgets.HBox([author_input, title_input, tissue_input]),
    widgets.HBox([geo_year_start, geo_year_end, geo_sort]),
    widgets.HBox([geo_limit, geo_cap]),
    geo_button, geo_output
]))


In [ ]:
to_annotate = load_candidates_for_annotation(f"{project_dir}/results/gradio_search.json")
label_ui.launch()


In [ ]:
#@title 可视化标注界面
import gradio as gr

# 用于标注的候选数据（你可以从搜索结果中加载）
to_annotate = []  # 结构: [{"features": [...], "title": ..., "accession": ..., ...}]
annotations = []

def load_candidates_for_annotation(json_path):
    import json
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    for item in data:
        item["features"] = extract_features(item, target={"title": "", "expected_tissue": ""})
    return data

def annotate(index, label):
    if 0 <= index < len(to_annotate):
        feat = to_annotate[index]["features"]
        annotations.append({"features": feat, "label": int(label)})
        return f"✅ 已标注第 {index} 条为 {'正例' if label else '负例'}"
    return "❌ 索引无效"

with gr.Blocks() as label_ui:
    gr.Markdown("### 🏷️ 手动标注候选数据")
    idx = gr.Number(label="候选索引", value=0)
    label = gr.Radio([1, 0], label="是否为正确匹配？", value=1)
    btn = gr.Button("提交标注")
    out = gr.Textbox()
    btn.click(fn=annotate, inputs=[idx, label], outputs=out)

# 启动标注界面（运行前请先加载 to_annotate）
# 示例：to_annotate = load_candidates_for_annotation(f"{project_dir}/results/gradio_search.json")
# label_ui.launch()


In [ ]:
#@title 模型训练与保存
from sklearn.linear_model import LogisticRegression
import joblib
import csv
import json
from pathlib import Path

def train_model_from_annotations(annotations):
    X = [a["features"] for a in annotations]
    y = [a["label"] for a in annotations]
    model = LogisticRegression()
    model.fit(X, y)
    return model

def save_annotations_json(data, path):
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

def save_annotations_csv(data, path):
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["bm25", "tissue_hit", "is_mouse", "is_human", "year_diff", "is_live", "label"])
        for item in data:
            writer.writerow(item["features"] + [item["label"]])

def save_model(model, path):
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    joblib.dump(model, path)
    print(f"✅ 模型已保存到: {path}")

# 示例保存
save_annotations_json(annotations, f"{project_dir}/annotations/annotations.json")
save_annotations_csv(annotations, f"{project_dir}/annotations/annotations.csv")

# 训练并保存模型
model = train_model_from_annotations(annotations)
save_model(model, f"{project_dir}/models/logreg_model.pkl")


In [ ]:
#@title 加载模型并对新数据打分排序
def load_model(path):
    return joblib.load(path)

def score_candidates(candidates, target, model, paper_year=None):
    X = [extract_features(c, target, paper_year) for c in candidates]
    scores = model.predict_proba(X)[:, 1]
    for c, s in zip(candidates, scores):
        c["score"] = round(float(s), 4)
    return sorted(candidates, key=lambda x: x["score"], reverse=True)

# 示例加载模型并打分
model = load_model(f"{project_dir}/models/logreg_model.pkl")

# 加载新候选数据
with open(f"{project_dir}/results/gradio_search.json", "r", encoding="utf-8") as f:
    new_candidates = json.load(f)

# 添加 features 并打分
for c in new_candidates:
    c["features"] = extract_features(c, target={"title": "", "expected_tissue": ""})

ranked = score_candidates(new_candidates, target={"title": "", "expected_tissue": ""}, model=model)

# 保存打分结果
df = pd.DataFrame(ranked)
df.to_csv(f"{project_dir}/results/ranked_candidates.csv", index=False)
print(f"✅ 打分结果已保存到: {project_dir}/results/ranked_candidates.csv")
